# Question-Agnostic Compression Tutorial

Learn how to compress context **without needing a specific question**.

## 📚 Available Model

| Model | Type | Description | Compression Ratio | Speed | Cost |
|-------|------|-------------|-------------------|-------|------|
| **agnostic_compressor_1** | LLMLingua-2 | Intelligently removes less important tokens | ✅ 0.1-0.9 | ~100-200ms | $0.20/1M tokens |

**When to use**: General purpose compression, system prompts, documentation - when you don't have a specific query.

## 1. Installation

In [28]:
# Install from local source in editable mode
%pip install -e ../ python-dotenv

Obtaining file:///Users/oussama/Documents/Cmprsr/codebase/Compresr-SDK-Private/python
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for compresr (pyproject.toml) ... done
  Created wheel for compresr: filename=compresr-1.1.1-0.editable-py3-none-any.whl size=5409 sha256=f0c050c75839b518d75bf694fc06f9ad90db408fe0d3f579b588360fe1f44d52
  Stored in directory: /private/var/folders/g4/rzc_8mgd15n5f0l73k0mm3600000gn/T/pip-ephem-wheel-cache-0n6xjdi1/wheels/b8/66/33/38ffb0bd34b6589c6f5f9d07abdc3fcffb34736427d2cdac72
Successfully built compresr
  Attempting uninstall: compresr
    Found existing installation: compresr 1.1.1
    Uninstalling compresr-1.1.1:
      Successfully uninstalled compresr-1.1.1
Note: you may need to restart the kernel to use updated packages.


## 2. Setup

In [29]:
import os
from dotenv import load_dotenv
from compresr import AgnosticCompressor, MODELS

# Load environment variables
env_path = "../../.env"
load_dotenv(env_path)

api_key = os.getenv("COMPRESR_API_KEY")

# For local testing
os.environ["COMPRESR_BASE_URL"] = "http://localhost:8000"

# Initialize client
client = AgnosticCompressor(api_key=api_key)

print("✅ Client initialized!")
print(f"🌐 Base URL: {os.getenv('COMPRESR_BASE_URL', 'https://api.compresr.ai')}")
print(f"\n📚 Agnostic Model: {MODELS.AGNOSTIC_LINGUA}")

✅ Client initialized!
🌐 Base URL: http://localhost:8000

📚 Agnostic Model: agnostic_compressor_1


## 3. Basic Compression

In [ ]:
context = """
Artificial intelligence (AI) represents one of the most transformative technological 
advancements of the 21st century. At its core, AI encompasses a broad range of 
computational techniques designed to enable machines to perform tasks that typically 
require human intelligence, including visual perception, speech recognition, 
decision-making, language translation, and pattern recognition.

Machine learning (ML), a crucial subset of AI, provides systems with the ability to 
automatically learn from experience without being explicitly programmed. ML algorithms 
analyze vast amounts of data, identify patterns, and build models that can make 
predictions on new data.

Deep learning uses artificial neural networks with multiple layers to model complex 
patterns. These networks excel at tasks involving images, audio, text, and time-series 
data, achieving superhuman performance in domains like image classification and gaming.
"""

target_ratio = 0.5
print("🤖 Model: agnostic_compressor_1 (LLMLingua-2)")
print("📝 Compressing without a specific question...\n")

response = client.compress(
    context=context.strip(),
    compression_model_name=MODELS.AGNOSTIC_LINGUA,
    target_compression_ratio=target_ratio
)

print("✅ Compression successful!")
print("\n📊 Results:")
print(f"   Original tokens: {response.data.original_tokens}")
print(f"   Compressed tokens: {response.data.compressed_tokens}")
print(f"   Tokens saved: {response.data.tokens_saved}")
print(f"   Target ratio: {target_ratio:.1%}")
print(f"   Actual ratio: {response.data.actual_compression_ratio:.1%}")
print("\n📝 Compressed context:")
print(response.data.compressed_context)

🤖 Model: agnostic_compressor_1 (LLMLingua-2)
📝 Compressing without a specific question...

✅ Compression successful!

📊 Results:
   Original tokens: 166
   Compressed tokens: 81
   Tokens saved: 85
   Target ratio: 50.0%
   Actual ratio: 51.2%

📝 Compressed context:
Artificial) transformative technological
 21st century. encompasses
 computational techniques machines tasks
 human visual perception speech recognition
 decision-making language translation pattern recognition. Machine learning provides systems
 learn experience without programmed. algorithms
 analyze data identify patterns build models
 predictions new data. Deep learning uses neural networks model complex
 patterns. networks excel tasks images audio text time-series
 data superhuman performance image classification gaming.


## 4. Controlling Compression with target_compression_ratio

- **ratio = 0.2** → Keep 20%, remove 80% (aggressive compression)
- **ratio = 0.5** → Keep 50%, remove 50% (medium compression)  
- **ratio = 0.8** → Keep 80%, remove 20% (light compression)

In [31]:
print("🎯 Testing Different Compression Ratios\n")

for ratio in [0.2, 0.5, 0.8]:
    response = client.compress(
        context=context.strip(),
        compression_model_name=MODELS.AGNOSTIC_LINGUA,
        target_compression_ratio=ratio
    )
    
    print(f"{'='*60}")
    print(f"Target Ratio {ratio:.1f} (keep {ratio*100:.0f}%, remove {(1-ratio)*100:.0f}%)")
    print(f"{'='*60}")
    print(f"Tokens: {response.data.original_tokens} → {response.data.compressed_tokens}")
    print(f"Actual Compression Ratio: {response.data.actual_compression_ratio:.1%}")
    print(f"Compressed: {response.data.compressed_context[:150]}...\n")

🎯 Testing Different Compression Ratios

Target Ratio 0.2 (keep 20%, remove 80%)
Tokens: 166 → 128
Actual Compression Ratio: 22.9%
Compressed: Artificial intelligence (AI) represents transformative technological
 advancements of 21st century. core, AI encompasses broad range
 computational te...

Target Ratio 0.5 (keep 50%, remove 50%)
Tokens: 166 → 81
Actual Compression Ratio: 51.2%
Compressed: Artificial) transformative technological
 21st century. encompasses
 computational techniques machines tasks
 human visual perception speech recogniti...

Target Ratio 0.8 (keep 80%, remove 20%)
Tokens: 166 → 35
Actual Compression Ratio: 78.9%
Compressed: 21st.

 visual perception speech
 decision-making language translation pattern recognition. learning
.
 analyze data patterns
. networks
. images audi...



## 5. Direct HTTP API (Without SDK)

In [32]:
import httpx

base_url = os.getenv("COMPRESR_BASE_URL", "https://api.compresr.ai")
url = f"{base_url}/api/compress/question-agnostic/"
headers = {"Content-Type": "application/json", "X-API-Key": api_key}
target_ratio = 0.5
payload = {
    "context": context.strip(),
    "compression_model_name": "agnostic_compressor_1",
    "target_compression_ratio": target_ratio
}

print(f"📡 POST request to {url}...\n")
response_http = httpx.post(url, headers=headers, json=payload)

if response_http.status_code == 200:
    data = response_http.json()
    print("✅ HTTP request successful!")
    print(f"   Original: {data['data']['original_tokens']} tokens")
    print(f"   Compressed: {data['data']['compressed_tokens']} tokens")
    print(f"   Target ratio: {target_ratio:.1%}")
    print(f"   Actual ratio: {data['data']['actual_compression_ratio']:.1%}")
else:
    print(f"❌ Error: {response_http.status_code}")

📡 POST request to http://localhost:8000/api/compress/question-agnostic/...

✅ HTTP request successful!
   Original: 166 tokens
   Compressed: 81 tokens
   Target ratio: 50.0%
   Actual ratio: 51.2%


## 6. Batch Compression

In [33]:
contexts = [
    "Machine learning enables systems to learn from data without explicit programming. It powers recommendations, fraud detection, and autonomous vehicles.",
    "Deep learning uses multi-layer neural networks to model complex patterns. CNNs excel at image processing while RNNs handle sequential data.",
    "Natural language processing enables computers to understand human language. Modern NLP uses transformers like BERT and GPT for various tasks.",
]

target_ratio = 0.5
response = client.compress_batch(
    contexts=contexts,
    compression_model_name=MODELS.AGNOSTIC_LINGUA,
    target_compression_ratio=target_ratio,
)

print(f"✅ Compressed {len(response.data.results)} contexts")
print(f"   Target ratio: {target_ratio:.1%}")
print(f"   Total tokens saved: {response.data.total_tokens_saved}")
print(f"   Average actual ratio: {response.data.average_compression_ratio:.1%}\n")

for i, result in enumerate(response.data.results):
    print(f"📝 Context {i+1}:")
    print(f"   Compressed: {result.compressed_context[:80]}...")
    print(f"   Saved: {result.tokens_saved} tokens")
    print(f"   Actual ratio: {result.actual_compression_ratio:.1%}\n")

✅ Compressed 3 contexts
   Target ratio: 50.0%
   Total tokens saved: 37
   Average actual ratio: 50.0%

📝 Context 1:
   Compressed: learning programming. powers recommendations fraud detection autonomous vehicles...
   Saved: 13 tokens
   Actual ratio: 56.5%

📝 Context 2:
   Compressed: learning multi-layer networks patterns. CNNs image handle sequential data....
   Saved: 13 tokens
   Actual ratio: 50.0%

📝 Context 3:
   Compressed: language processing computers understand. NLP uses transformers BERT GPT....
   Saved: 11 tokens
   Actual ratio: 44.0%



## 7. Async Compression

In [ ]:
async def compress_async():
    context = "Quantum computing uses qubits that can exist in superposition. This enables parallel computation on an exponential scale, potentially solving problems classical computers cannot."
    target_ratio = 0.6
    
    response = await client.compress_async(
        context=context,
        compression_model_name=MODELS.AGNOSTIC_LINGUA,
        target_compression_ratio=target_ratio
    )
    print("✅ Async compression successful!")
    print(f"   Tokens: {response.data.original_tokens} → {response.data.compressed_tokens}")
    print(f"   Saved: {response.data.tokens_saved} tokens")
    print(f"   Target ratio: {target_ratio:.1%}")
    print(f"   Actual ratio: {response.data.actual_compression_ratio:.1%}")
    print(f"\n📝 Compressed: {response.data.compressed_context}")

await compress_async()

✅ Async compression successful!
   Tokens: 29 → 13
   Saved: 16 tokens
   Target ratio: 60.0%
   Actual ratio: 55.2%

📝 Compressed: Quantum qubits superposition. enables parallel computation exponential scale.


## 8. Error Handling

In [35]:
from compresr.exceptions import (
    CompresrError,
    AuthenticationError,
    RateLimitError,
    ValidationError
)

# Example 1: Successful compression
try:
    response = client.compress(
        context="Test context",
        compression_model_name=MODELS.AGNOSTIC_LINGUA,
    )
    print("✅ Success!")
except AuthenticationError:
    print("❌ Invalid API key")
except RateLimitError:
    print("⏳ Rate limit exceeded")
except ValidationError as e:
    print(f"❌ Invalid input: {e}")
except CompresrError as e:
    print(f"❌ API error: {e}")

✅ Success!


### Common Error Scenarios

Test how the SDK handles various error conditions:

In [ ]:
from compresr import AgnosticCompressor
# Error 1: Unsupported model name
print("1️⃣  Testing unsupported model name...")
try:
    response = client.compress(
        context="Test context",
        compression_model_name="invalid_model_v1",  # ❌ Invalid model
    )
    print("   ❌ Unexpected success - should have raised an error\n")
except ValidationError as e:
    print(f"   ✅ Caught ValidationError: {e}\n")
except Exception as e:
    print(f"   ⚠️  Caught {type(e).__name__}: {e}\n")

# Error 2: Empty context
print("2️⃣  Testing empty context...")
try:
    response = client.compress(
        context="",  # ❌ Empty context
        compression_model_name=MODELS.AGNOSTIC_LINGUA,
    )
    print("   ❌ Unexpected success - should have raised an error\n")
except ValidationError as e:
    print(f"   ✅ Caught ValidationError: {e}\n")
except Exception as e:
    print(f"   ⚠️  Caught {type(e).__name__}: {e}\n")

# Error 3: Invalid compression ratio (out of range)
print("3️⃣  Testing invalid compression ratio...")
try:
    response = client.compress(
        context="Test context",
        compression_model_name=MODELS.AGNOSTIC_LINGUA,
        target_compression_ratio=1.5,  # ❌ Must be 0.0-1.0
    )
    print("   ❌ Unexpected success - should have raised an error\n")
except ValidationError as e:
    print(f"   ✅ Caught ValidationError: {e}\n")
except Exception as e:
    print(f"   ⚠️  Caught {type(e).__name__}: {e}\n")

# Error 4: Invalid API key format (caught at client creation)
print("4️⃣  Testing invalid API key format...")
try:
    bad_client = AgnosticCompressor(api_key="sk-invalid-key")  # ❌ Wrong prefix
    response = bad_client.compress(
        context="Test context",
        compression_model_name=MODELS.AGNOSTIC_LINGUA,
    )
    print("   ❌ Unexpected success - should have raised an error\n")
except AuthenticationError as e:
    print(f"   ✅ Caught AuthenticationError: {e}\n")
except Exception as e:
    print(f"   ⚠️  Caught {type(e).__name__}: {e}\n")

# Error 5: Using QS model with AgnosticCompressor (wrong client)
print("5️⃣  Testing wrong model for client type...")
try:
    response = client.compress(
        context="Test context",
        compression_model_name=MODELS.QS_GEMFILTER,  # ❌ QS model with Agnostic client
    )
    print("   ❌ Unexpected success - should have raised an error\n")
except ValidationError as e:
    print(f"   ✅ Caught ValidationError: {e}\n")
except Exception as e:
    print(f"   ⚠️  Caught {type(e).__name__}: {e}\n")

print("✅ All error scenarios tested!")

1️⃣  Testing unsupported model name...
   ✅ Caught ValidationError: Model 'invalid_model_v1' is not valid for AgnosticCompressor. Allowed models: agnostic_compressor_1

2️⃣  Testing empty context...
   ⚠️  Caught ServerError: Server error: Internal server error. Please try again later or contact support.

3️⃣  Testing invalid compression ratio...
   ✅ Caught ValidationError: 1 validation error for CompressRequest
target_compression_ratio
  Input should be less than or equal to 0.9 [type=less_than_equal, input_value=1.5, input_type=float]
    For further information visit https://errors.pydantic.dev/2.12/v/less_than_equal

4️⃣  Testing invalid API key format...
   ✅ Caught AuthenticationError: Invalid API key format. Keys must start with 'cmp_'

5️⃣  Testing wrong model for client type...
   ✅ Caught ValidationError: Model 'qs_gemfilter_v1' is not valid for AgnosticCompressor. Allowed models: agnostic_compressor_1

✅ All error scenarios tested!


## Next Steps

- 📖 Docs: [docs.compresr.ai](https://docs.compresr.ai)
- 💬 Support: founders@compresr.ai
- 🚀 Try question-specific compression: `question_specific_compression_tutorial.ipynb`